In [1]:
cd ..

/home/abdalrhman/Desktop/Graduation Project/AiStore/Recommender_V1


# 1. Setup & Imports

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch_geometric.data import Data
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import negative_sampling
from srcs.utils.logger import get_module_logger
from srcs.utils.settings import (
    CLEANED_REVIEWS_PATH_CSV,
    CLEANED_METADATA_PATH_CSV,
    FULL_GRAPH_PATH,
    TRAIN_GRAPH_PATH,
    VAL_GRAPH_PATH,
    TEST_GRAPH_PATH,
    IMAGES_DIR,
    GNN_MODEL_SAVE_PATH,
    PREDICTOR_MODEL_SAVE_PATH
)

# Configure paths
NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, os.pardir, os.pardir))
sys.path.append(PROJECT_ROOT)

# Initialize logger
logger = get_module_logger("graph_builder")

/home/abdalrhman/Desktop/Graduation Project/AiStore/venv/lib/python3.12/site-packages/torch/__config__.py:10: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._show_config()


In [3]:
import torch
from torch_geometric.data import HeteroData


# Load the full graph
# full_data = torch.load(FULL_GRAPH_PATH)

# Load train, validation, and test splits
train_data = torch.load(TRAIN_GRAPH_PATH, weights_only=False)
val_data = torch.load(VAL_GRAPH_PATH, weights_only=False)
test_data = torch.load(TEST_GRAPH_PATH, weights_only=False)

# Optional: Move data to device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# full_data = full_data.to(device)
train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import SAGEConv, HeteroConv, Linear
from torch_geometric.data import HeteroData
from torch_geometric.transforms import RandomLinkSplit
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, precision_score, recall_score
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ─── 1) Data Preparation ───────────────────────────────────────────────
transform = RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    is_undirected=True,
    add_negative_train_samples=False,
    edge_types=[('user', 'rates', 'item')],
    rev_edge_types=[('item', 'rev_rates', 'user')]
)

train_data, val_data, test_data = transform(data)
train_data, val_data, test_data = [d.to(device) for d in (train_data, val_data, test_data)]

num_items = train_data['item'].x.size(0)
edge_set = set(map(tuple, train_data['user','rates','item'].edge_index.cpu().t().tolist()))

# ─── 2) Fixed Heterogeneous GraphSAGE Model ────────────────────────────
class HeteroGraphSAGE(nn.Module):
    def __init__(self, hidden_dim=128):
        super().__init__()
        self.conv1 = HeteroConv({
            ('user', 'rates', 'item'): SAGEConv((-1, -1), hidden_dim),
            ('item', 'rev_rates', 'user'): SAGEConv((-1, -1), hidden_dim)
        }, aggr='mean')
        
        self.conv2 = HeteroConv({
            ('user', 'rates', 'item'): SAGEConv(hidden_dim, hidden_dim),
            ('item', 'rev_rates', 'user'): SAGEConv(hidden_dim, hidden_dim)
        }, aggr='mean')
        
        # BatchNorm for each node type
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        
        # Dropout (now applied per node type)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x_dict, edge_index_dict):
        # First conv layer
        x_dict = self.conv1(x_dict, edge_index_dict)
        x_dict = {key: self.bn1(x) for key, x in x_dict.items()}
        x_dict = {key: F.leaky_relu(x) for key, x in x_dict.items()}
        x_dict = {key: self.dropout(x) for key, x in x_dict.items()}  # ✅ Fixed: Apply dropout per tensor

        # Second conv layer
        x_dict = self.conv2(x_dict, edge_index_dict)
        x_dict = {key: self.bn2(x) for key, x in x_dict.items()}

        return x_dict

# ─── 3) Link Predictor with MLP ───────────────────────────────────────
class LinkPredictor(nn.Module):
    def __init__(self, hidden_dim=128):
        super().__init__()
        self.mlp = nn.Sequential(
            Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            Linear(hidden_dim, 1)
        )

    def forward(self, src, dst):
        return self.mlp(torch.cat([src, dst], dim=-1)).squeeze()

# ─── 4) Initialize Model, Optimizer, Scheduler ───────────────────────
hidden_dim = 128
gnn = HeteroGraphSAGE(hidden_dim).to(device)
predictor = LinkPredictor(hidden_dim).to(device)

optimizer = torch.optim.AdamW([
    {'params': gnn.parameters(), 'lr': 1e-3},
    {'params': predictor.parameters(), 'lr': 5e-4}
], weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=5, verbose=True
)

NameError: name 'data' is not defined

In [ ]:
# ─── 5) Training with Hard Negative Sampling ─────────────────────────
def train_epoch(data, neg_ratio=5):
    gnn.train()
    predictor.train()
    optimizer.zero_grad()

    emb = gnn(data.x_dict, data.edge_index_dict)

    edge_index = data['user', 'rates', 'item'].edge_index
    src, dst = edge_index[0], edge_index[1]
    pos_pred = predictor(emb['user'][src], emb['item'][dst])

    # Hard negative sampling
    neg_src, neg_dst = [], []
    for u in src.unique().cpu().tolist():
        seen = dst[src == u]
        candidates = torch.randint(0, num_items, (len(seen)*neg_ratio,), device=seen.device)
        valid = ~torch.isin(candidates, seen)
        sampled = candidates[valid][:len(seen)]
        if len(sampled) > 0:
            neg_src.append(torch.tensor([u]*len(sampled), device=device))
            neg_dst.append(sampled)

    if not neg_src:
        return 0.0

    neg_src = torch.cat(neg_src)
    neg_dst = torch.cat(neg_dst)

    neg_pred = predictor(emb['user'][neg_src], emb['item'][neg_dst])

    loss = F.binary_cross_entropy_with_logits(
        torch.cat([pos_pred, neg_pred]),
        torch.cat([torch.ones_like(pos_pred), torch.zeros_like(neg_pred)])
    )

    loss.backward()
    optimizer.step()
    return loss.item()

# ─── 6) Evaluation with Dynamic Threshold ───────────────────────────
@torch.no_grad()
def evaluate(data):
    gnn.eval()
    predictor.eval()
    emb = gnn(data.x_dict, data.edge_index_dict)

    edge_index = data['user', 'rates', 'item'].edge_index
    src, dst = edge_index[0], edge_index[1]
    
    pos_pred = predictor(emb['user'][src], emb['item'][dst]).cpu().numpy()
    pos_labels = np.ones(len(pos_pred))

    neg_src, neg_dst = [], []
    for u in src.unique().cpu().tolist():
        seen = dst[src == u]
        candidates = torch.randint(0, num_items, (len(seen)*5,), device=seen.device)
        valid = ~torch.isin(candidates, seen)
        sampled = candidates[valid][:len(seen)]
        if len(sampled) > 0:
            neg_src.append(torch.tensor([u]*len(sampled), device=device))
            neg_dst.append(sampled)

    if not neg_src:
        return 0.0, 0.0, 0.0, 0.0, 0.0

    neg_pred = predictor(emb['user'][torch.cat(neg_src)], emb['item'][torch.cat(neg_dst)]).cpu().numpy()
    neg_labels = np.zeros(len(neg_pred))

    scores = np.concatenate([pos_pred, neg_pred])
    labels = np.concatenate([pos_labels, neg_labels])

    auc = roc_auc_score(labels, scores)
    prec, rec, threshs = precision_recall_curve(labels, scores)
    f1s = 2 * (prec * rec) / (prec + rec + 1e-8)
    best_idx = np.argmax(f1s)
    best_thresh = threshs[best_idx]

    preds = (scores >= best_thresh).astype(int)
    f1 = f1_score(labels, preds)
    prec_final = precision_score(labels, preds)
    rec_final = recall_score(labels, preds)

    return {
        'auc': auc,
        'f1': f1,
        'precision': prec_final,
        'recall': rec_final,
        'threshold': best_thresh
    }

In [ ]:
# ─── 7) Training Loop with Early Stopping ───────────────────────────
best_f1 = 0.0
patience = 0

for epoch in range(1, 100):
    loss = train_epoch(train_data)
    val_metrics = evaluate(val_data)
    scheduler.step(val_metrics['f1'])

    print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | "
          f"Val AUC: {val_metrics['auc']:.4f} | F1: {val_metrics['f1']:.4f} | "
          f"Prec: {val_metrics['precision']:.4f} | Rec: {val_metrics['recall']:.4f}")

    if val_metrics['f1'] > best_f1:
        best_f1 = val_metrics['f1']
        torch.save(gnn.state_dict(), 'best_gnn.pth')
        torch.save(predictor.state_dict(), 'best_predictor.pth')
        patience = 0
    else:
        patience += 1
        if patience >= 20:
            print("Early stopping triggered.")
            break

# ─── 8) Final Test Evaluation ───────────────────────────────────────
gnn.load_state_dict(torch.load('best_gnn.pth'))
predictor.load_state_dict(torch.load('best_predictor.pth'))
test_metrics = evaluate(test_data)

print("\nTest Metrics:")
print(f"AUC: {test_metrics['auc']:.4f} | Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f} | F1: {test_metrics['f1']:.4f}")
print(f"Optimal Threshold: {test_metrics['threshold']:.4f}")

In [ ]:
# Save the best model after training
torch.save(gnn.state_dict(), GNN_MODEL_SAVE_PATH)
torch.save(predictor.state_dict(), PREDICTOR_MODEL_SAVE_PATH)

print("Models saved successfully.")

In [ ]:
# Recreate model architecture
hidden_dim = 128
gnn = HeteroGraphSAGE(hidden_dim)
predictor = LinkPredictor(hidden_dim)

# Load saved weights with map_location set to 'cpu'
gnn.load_state_dict(torch.load(GNN_MODEL_SAVE_PATH, map_location=torch.device('cpu')))
predictor.load_state_dict(torch.load(PREDICTOR_MODEL_SAVE_PATH, map_location=torch.device('cpu')))

# Move models to desired device (optional)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gnn = gnn.to(device)
predictor = predictor.to(device)

# Set models to evaluation mode for inference
gnn.eval()
predictor.eval()

In [ ]:
gnn.train()
predictor.train()

# 10. Evaluate the Model

# ✅ 1. Top-K Recommendations for a User

Recommend top k items for a given user based on predicted interaction scores.

In [ ]:
@torch.no_grad()
def recommend(user_id, gnn_model, predictor_model, data, k=10):
    """
    Recommend top-k items for a given user based on learned embeddings.
    
    Args:
        user_id (int): ID of the user
        gnn_model: Trained heterogeneous GNN model (HeteroGraphSAGE)
        predictor_model: Link predictor (MLP)
        data: HeteroData object (val or test split)
        k (int): Number of recommendations to return
    
    Returns:
        list: Top-k item IDs recommended for the user
    """
    gnn_model.eval()
    predictor_model.eval()

    # Get embeddings for all nodes
    emb = gnn_model(data.x_dict, data.edge_index_dict)

    # Get user embedding
    user_emb = emb['user'][user_id].unsqueeze(0)  # shape: [1, hidden_dim]

    # Get all item embeddings
    item_embs = emb['item']  # shape: [num_items, hidden_dim]

    # Repeat user embedding to match item_embs size
    user_embs = user_emb.expand(item_embs.size(0), -1)

    # Predict scores for all items
    scores = predictor_model(user_embs, item_embs).cpu().numpy()

    # Exclude already interacted items
    interacted_items = data['user', 'rates', 'item'].edge_index[1][
        data['user', 'rates', 'item'].edge_index[0] == user_id
    ].cpu().numpy()

    # Mask out interacted items
    scores[interacted_items] = -np.inf

    # Get top-k item indices
    top_k_item_ids = np.argpartition(scores, -k)[-k:]
    top_k_item_ids = top_k_item_ids[np.argsort(-scores[top_k_item_ids])]  # Sort descending

    return top_k_item_ids.tolist()

In [ ]:
user_id = 0
recommended_items = recommend(user_id, gnn, predictor, test_data, k=5)
print(f"Top-5 Recommended Items for User {user_id}: {recommended_items}")

# 2. Item-to-Item Recommendations (Similar Items)

In [ ]:
@torch.no_grad()
def similar_items(item_id, gnn_model, data, k=5):
    gnn_model.eval()
    emb = gnn_model(data.x_dict, data.edge_index_dict)['item']
    item_emb = emb[item_id].unsqueeze(0)
    
    # Compute cosine similarity
    sim = F.cosine_similarity(item_emb, emb).cpu().numpy()
    sim[item_id] = -np.inf  # Exclude self
    
    # Top-k most similar items
    top_k = np.argpartition(sim, -k)[-k:]
    return top_k[np.argsort(-sim[top_k])].tolist()

In [ ]:
similar_items(42, gnn, test_data, k=5)